# 転職者向け　ケーススタディコース

## Day1: Online Retail
https://archive.ics.uci.edu/ml/datasets/Online+Retail

ver1.0<br>
copyright 2019 DataMix Co., Ltd.

## 0. ライブラリのインポート

もしimportできないライブラリがあったら、pip installにてライブラリをダウンロードしましょう

In [1]:
import pandas as pd
import numpy as np
import pymysql
import pandas.io.sql as psql
import matplotlib.pyplot as plt
import scipy.spatial as sp
import time

## 1. データの読み込み

### データベースへ接続

In [2]:
con = pymysql.connect(
    host='datamix-school-material.csbsmnjyxb52.ap-northeast-1.rds.amazonaws.com',
    user='user1',
    password='user1',
    db='online_retail',
    charset='utf8mb4',
    cursorclass=pymysql.cursors.DictCursor)
con

### 【演習1】SQLによるデータ抽出
<b>各顧客（CustomerID）に対して、それぞれの商品（StockCode）を、何個（Quantity）卸したのかを集計してみましょう！</b>

- 必要なカラムはなにか？
- 除外しておいた方が良いデータはないか？

In [14]:
sql = """
SELECT *
-- SELECT InvoiceNo, StockCode, Quantity, CustomerID, Country 
FROM sales_log
WHERE Quantity > 0 and UnitPrice > 0
"""

In [15]:
data = psql.read_sql(con=con, sql=sql)

In [17]:
data = data[data["CustomerID"]!=""]
data = data[data["Description"]!=""]
data = data[["InvoiceNo", "StockCode", "Quantity", "CustomerID", "Country"]]

In [5]:
data.head()

,InvoiceNo,StockCode,Quantity,CustomerID,Country
0,536365,85123A,6,17850,United Kingdom
1,536365,71053,6,17850,United Kingdom
2,536365,84406B,8,17850,United Kingdom
3,536365,84029G,6,17850,United Kingdom
4,536365,84029E,6,17850,United Kingdom


In [20]:
data.shape

(168662, 5)

## 2. 顧客ID×商品コードのテーブルを作成

### 【演習2】Pandasを使ったデータ加工
<b>Pandasを使って、顧客ID（CustomerID）×商品コード（StockCode）のテーブルを作成しましょう</b>

- Pandasのどの関数を使ったら良いか？
- テーブルにおける、行・列・行列内の値、は何を指定すれば良いか？

In [ ]:
customer_stock_matrix = 

In [ ]:
customer_stock_matrix.head()

In [ ]:
customer_stock_matrix.shape

## 3. レコメンデーション用の関数を作成

### 【演習3】レコメンドする関数を作成しましょう

<b>ある顧客に対して、新たな商品をレコメンデーションする関数を完成させましょう</b>

- 準備(Pandas Dataframeから、index, columns, 行列を取得しておく)

In [ ]:
stock_list = customer_stock_matrix.columns
customer_list = customer_stock_matrix.index
customer_stock_matrix_array = np.array(customer_stock_matrix)

print('description of each stock: \n', stock_list[0:5])
print('\ncustomer id: \n', customer_list[0:5])
print('\ncustomer id - stock description matrix: \n', customer_stock_matrix_array[0:5])

- あるレコメンドしたい顧客IDを引数とした際に、顧客リスト（customer_list）から、レコメンドしたい顧客のindexを取り出す関数

In [ ]:
def get_customer_list_index(customer_list, target_customer_id):
    for index in range(len(customer_list)):
        if customer_list[index] == target_customer_id:
            target_customer_index = index
    return target_customer_index

In [ ]:
# サンプル
target_customer_id = '12346'
target_customer_index = get_customer_list_index(customer_list, target_customer_id)
target_customer_index

- 二つのベクトルのコサイン類似度を計算する関数

In [ ]:
def get_cosine_similarity(x, y):  
    # cosine類似度　= 1 - cosine距離
    return 1- sp.distance.cosine(x, y)

In [ ]:
# サンプル
x = [0.1, 0.5, 0.9, 0]
y = [0, 0.35, 0.65, 1]
get_cosine_similarity(x, y)

- レコメンドしたい顧客と、全顧客との類似度を計算する関数

In [ ]:
def get_user_similarity(customer_stock_matrix_array, target_customer_index):
    user_similarity = []
    target_user_row = customer_stock_matrix_array[target_customer_index]

    for row in customer_stock_matrix_array:
        similarity = get_cosine_similarity(target_user_row, row)
        user_similarity.append(similarity)

    user_similarity = np.array(user_similarity)
    return user_similarity

In [ ]:
user_similarity = get_user_similarity(customer_stock_matrix_array, target_customer_index)
user_similarity

- レコメンドしたい顧客に対してコサイン類似度の高いユーザーTopN人を対象に、商品の平均類似度を計算する関数

In [ ]:
def get_average_score_of_item(user_similarity, customer_stock_matrix_array, user_top_N):
    # TopN人の類似度と購入数量行列を取得
    idx = user_similarity.argsort()[::-1][1:user_top_N+1]
    selected_user_similarity = user_similarity[idx]
    selected_matrix = customer_stock_matrix_array[idx]
    
    # ユーザーTopN人を対象に、商品ごとの平均類似度を計算
    avg_score = []
    for col_idx in range(selected_matrix.shape[1]):
        weight_score = sum(selected_matrix[:, col_idx] * selected_user_similarity)
        similarity_sum = sum(selected_user_similarity[selected_user_similarity > 0])
        avg_score.append(weight_score / similarity_sum)
    avg_score = np.array(avg_score)
    return avg_score

In [ ]:
average_score_of_item = get_average_score_of_item(user_similarity, customer_stock_matrix_array, user_top_N=20)
average_score_of_item

In [ ]:
# 商品の平均類似度のヒストグラム
import matplotlib.pyplot as plt
plt.hist(average_score_of_item, bins=50, range=(0, 4))
plt.show()

-  レコメンドする必要がない、既に購入済の商品リストを抽出

In [ ]:
already_ordered_product = stock_list[np.where(customer_stock_matrix[customer_stock_matrix.index == target_customer_id])[1]]
already_ordered_product

- 平均類度の高い上位N商品をオススメとして表示する関数

In [ ]:
def print_recommendation_items(stock_list, average_score_of_item, already_ordered_product, recommend_top_N):
    counter = 0
    print("■レコメンド商品リスト:")
    for recommended_product in stock_list[average_score_of_item.argsort()[::-1]]:
        if recommended_product not in already_ordered_product:
            print(counter+1,recommended_product)
            counter += 1
            if recommend_top_N <= counter:
                break

In [ ]:
print_recommendation_items(stock_list, average_score_of_item, already_ordered_product, recommend_top_N=10)

### 【演習3】レコメンドをする関数を作成しましょう

<b>上記関数やコードをまとめて、ある顧客に対して、新たな商品をレコメンデーションする1つの関数を完成させ、実際に関数を実行させましょう</b>

(目安時間: 20分)

In [ ]:
def recommend(target_customer_id, customer_stock_matrix, user_top_N, recommend_top_N):
    pass

In [1]:
# 関数を実行させましょう (Enter your code)
